## Neural networks

In [2]:
%cd C:\\Users\\s\\Downloads\\machine_learning\\machine-learning-ex4\\ex4
%pwd

C:\Users\s\Downloads\machine_learning\machine-learning-ex4\ex4


'C:\\Users\\s\\Downloads\\machine_learning\\machine-learning-ex4\\ex4'

### Feedforward and cost function

In [44]:
import numpy as np
from scipy import optimize
from scipy.io import loadmat


data = loadmat('ex4data1.mat')
x0 = data['X']
y0 = data['y']
print(x0.shape, y0.shape)

weights = loadmat('ex4weights.mat')
theta1 = weights['Theta1']
theta2 = weights['Theta2']
print(theta1.shape, theta2.shape)
theta0 = np.hstack((theta1.ravel(), theta2.ravel()))


def sigmoid(z):
    g = 1 / (1 + np.exp(-z))
    
    return g


def nn_cost_function(theta, input_layer_size, hidden_layer_size, num_labels, x, y, lam):
    m = x.shape[0]
    
    weights1 = theta[:hidden_layer_size*(input_layer_size+1)].reshape(hidden_layer_size, input_layer_size+1)
    weights2 = theta[hidden_layer_size*(input_layer_size+1):].reshape(num_labels, hidden_layer_size+1)
    
    y_i = np.zeros((m, num_labels))
    for i in range(1, num_labels+1):
        y_i[:, i-1] = np.array([1 if label == i else 0 for label in y])
    
    a1 = np.hstack((np.ones((m, 1)), x))
    z2 = a1.dot(weights1.T)
    a2 = sigmoid(z2)
    a2 = np.hstack((np.ones((m, 1)), a2))
    z3 = a2.dot(weights2.T)
    a3 = sigmoid(z3)
    
    cost = y_i * np.log(a3) + (1 - y_i) * np.log(1 - a3)
    j = -1 / m * np.sum(cost)
    
    reg1 = weights1[:, 1:].ravel().dot(weights1[:, 1:].ravel().T)
    reg2 = weights2[:, 1:].ravel().dot(weights2[:, 1:].ravel().T)
    reg = lam / (2 * m) * (reg1 + reg2)
    
    j += reg
    
    return j


cost0 = nn_cost_function(theta0, 400, 25, 10, x0, y0, 1)
print(cost0)
    

(5000, 400) (5000, 1)
(25, 401) (10, 26)
0.38376985909092365


### Sigmoid gradient

In [26]:
def sigmoid_gradient(z):
    dg = sigmoid(z) * (1 - sigmoid(z))
    
    return dg


print(sigmoid_gradient(0))    

0.25


### Random initialization

In [27]:
def rand_initialize_weights(l_in, l_out):
    epsilon_init = 0.12
    w = np.random.rand(l_out, 1+l_in) * 2 * epsilon_init - epsilon_init
    
    return w

### Backpropagation

In [31]:
def nn_gradient(theta, input_layer_size, hidden_layer_size, num_labels, x, y, lam):
    m = x.shape[0]
    
    weights1 = theta[:hidden_layer_size*(input_layer_size+1)].reshape(hidden_layer_size, input_layer_size+1)
    weights2 = theta[hidden_layer_size*(input_layer_size+1):].reshape(num_labels, hidden_layer_size+1)
    
    y_i = np.zeros((m, num_labels))
    for i in range(1, num_labels+1):
        y_i[:, i-1] = np.array([1 if label == i else 0 for label in y])
    
    a1 = np.hstack((np.ones((m, 1)), x))
    z2 = a1.dot(weights1.T)
    a2 = sigmoid(z2)
    a2 = np.hstack((np.ones((m, 1)), a2))
    z3 = a2.dot(weights2.T)
    a3 = sigmoid(z3)
    
    delta3 = a3 - y_i
    delta2 = delta3.dot(weights2)
    delta2 = delta2[:, 1:]
    delta2 = delta2 * sigmoid_gradient(z2)
    
    grad1 = np.zeros(weights1.shape)
    grad2 = np.zeros(weights2.shape)

    grad1 += delta2.T.dot(a1)
    grad2 += delta3.T.dot(a2)
    
    t1_grad = 1 / m * grad1
    t2_grad = 1 / m *grad2
    
    reg_t1 = lam / m * weights1
    reg_t2 = lam / m * weights2
    reg_t1[:, 0] = np.zeros(reg_t1.shape[0])
    reg_t2[:, 0] = np.zeros(reg_t2.shape[0])
    
    t1_grad += reg_t1
    t2_grad += reg_t2
    
    grad = np.hstack((t1_grad.ravel(), t2_grad.ravel()))
    
    return grad


print(nn_gradient(theta0, 400, 25, 10, x0, y0, 1))

[ 6.18712766e-05 -2.11248326e-12  4.38829369e-13 ...  4.70513145e-05
 -5.01718610e-04  5.07825789e-04]


### Gradient checking

In [33]:
def compute_numerical_gradient(theta, input_layer_size, hidden_layer_size, num_labels, x, y, lam):
    num_grad = np.zeros(theta.shape)
    perturb = np.zeros(theta.shape)
    e = 1e-4
    
    for i in range(theta.shape[0]):
        perturb[i] = e
        loss1 = nn_cost_function(theta-perturb, input_layer_size, hidden_layer_size, num_labels, x, y, lam)
        loss2 = nn_cost_function(theta+perturb, input_layer_size, hidden_layer_size, num_labels, x, y, lam)
        num_grad[i] = (np.array(loss2) - np.array(loss1)) / (2 * e)
        perturb[i] = 0
        
    return num_grad


def debug_initialize_weights(fan_out, fan_in):
    w = np.zeros((fan_out, fan_in+1))
    w = np.reshape(np.sin(range(w.size)), w.shape) / 10
    
    return w


def check_nn_gradients(lam):
    input_layer_size = 3
    hidden_layer_size = 5
    num_labels = 3
    m = 5
    
    theta01 = debug_initialize_weights(hidden_layer_size, input_layer_size)
    theta02 = debug_initialize_weights(num_labels, hidden_layer_size)
    
    x = debug_initialize_weights(m, input_layer_size-1)
    y = 1 + np.mod([i+1 for i in range(m)], num_labels).T 
    theta = np.hstack((theta01.ravel(), theta02.ravel()))
    
    grad = nn_gradient(theta, input_layer_size, hidden_layer_size, num_labels, x, y, lam)
    num_grad = compute_numerical_gradient(theta, input_layer_size, hidden_layer_size, num_labels, x, y, lam)
    
    diff = max((num_grad - grad) / (num_grad + grad))
    
    return diff


print(check_nn_gradients(1))

6.300716890302574e-10


### Learning parameters using advanced optimization

In [39]:
params = np.hstack((rand_initialize_weights(400, 25).ravel(), rand_initialize_weights(25, 10).ravel()))
res = optimize.minimize(nn_cost_function, x0=params, args=(400, 25, 10, x0, y0, 1), method='CG', jac=nn_gradient, options={'maxiter':400})
res

     fun: 0.314143339290856
     jac: array([-2.97078025e-04, -4.65225049e-08,  4.09766973e-08, ...,
       -7.15493528e-05, -1.61146109e-04, -1.29273861e-04])
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 835
     nit: 400
    njev: 835
  status: 1
 success: False
       x: array([ 1.32908200e-01, -2.32612524e-04,  2.04883486e-04, ...,
       -1.87567530e+00,  1.41754530e+00,  3.91261043e+00])

### Check training result

In [50]:
param1 = res.x[:25 * 401].reshape(25, 401)
param2 = res.x[25 * 401:].reshape(10, 26)

a01 = np.hstack((np.ones((5000, 1)), x0))
a02 = sigmoid(a01.dot(param1.T))
a02 = np.hstack((np.ones((5000, 1)), a02))
a03 = sigmoid(a02.dot(param2.T))
y_pred = np.array(np.argmax(a03, axis=1) + 1)

print('Accuracy: {}%'.format(np.mean(y_pred.reshape(-1, 1) == y0) * 100))


Accuracy: 99.6%
